In [13]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [14]:
df = pd.read_csv(Path('./data/20200920.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,9/15/2020 21:15:50,Hunger (2008),First Reformed (2017),Resurrection Man (1998),Time Changer (2002),Ordet (1955),Winter Light (1963),THE KIDS AREN'T ALRIGHT,CLASSIC KUNG FU,STRICTLY ROMPS,BREAKING THE IRANIAN EMBARGO,CATHOLICS: FRIEND OR FOE?,NON-WESTERN SCIFI,No Theme/Free-for-all
1,9/15/2020 21:20:27,Ordet (1955),Winter Light (1963),Resurrection Man (1998),Time Changer (2002),NaN,NaN,BREAKING THE IRANIAN EMBARGO,CATHOLICS: FRIEND OR FOE?,THE KIDS AREN'T ALRIGHT,NON-WESTERN SCIFI,STRICTLY ROMPS,CLASSIC KUNG FU,No Theme/Free-for-all
2,9/16/2020 8:34:17,Winter Light (1963),Ordet (1955),First Reformed (2017),Hunger (2008),Resurrection Man (1998),Time Changer (2002),CATHOLICS: FRIEND OR FOE?,STRICTLY ROMPS,THE KIDS AREN'T ALRIGHT,NON-WESTERN SCIFI,CLASSIC KUNG FU,BREAKING THE IRANIAN EMBARGO,NaN
3,9/16/2020 17:22:55,Time Changer (2002),First Reformed (2017),Hunger (2008),Winter Light (1963),NaN,NaN,NON-WESTERN SCIFI,CLASSIC KUNG FU,BREAKING THE IRANIAN EMBARGO,CATHOLICS: FRIEND OR FOE?,STRICTLY ROMPS,NaN,NaN
4,9/16/2020 17:24:15,Winter Light (1963),Time Changer (2002),First Reformed (2017),Hunger (2008),Ordet (1955),Resurrection Man (1998),STRICTLY ROMPS,BREAKING THE IRANIAN EMBARGO,THE KIDS AREN'T ALRIGHT,CATHOLICS: FRIEND OR FOE?,CLASSIC KUNG FU,NON-WESTERN SCIFI,No Theme/Free-for-all
5,9/17/2020 10:32:14,Resurrection Man (1998),Hunger (2008),Winter Light (1963),First Reformed (2017),Ordet (1955),Time Changer (2002),THE KIDS AREN'T ALRIGHT,CLASSIC KUNG FU,STRICTLY ROMPS,NON-WESTERN SCIFI,BREAKING THE IRANIAN EMBARGO,CATHOLICS: FRIEND OR FOE?,No Theme/Free-for-all


In [15]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [16]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Hunger (2008)', 'First Reformed (2017)',
       'Resurrection Man (1998)', 'Time Changer (2002)', 'Ordet (1955)',
       'Winter Light (1963)'], dtype=object)

In [17]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [18]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [19]:
submission_dict

{'Hunger (2008)': <Candidate('Hunger (2008)')>,
 'First Reformed (2017)': <Candidate('First Reformed (2017)')>,
 'Resurrection Man (1998)': <Candidate('Resurrection Man (1998)')>,
 'Time Changer (2002)': <Candidate('Time Changer (2002)')>,
 'Ordet (1955)': <Candidate('Ordet (1955)')>,
 'Winter Light (1963)': <Candidate('Winter Light (1963)')>}

In [20]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [21]:
ballots

[<Ballot(Hunger (2008), First Reformed (2017), Resurrection Man (1998), Time Changer (2002), Ordet (1955), Winter Light (1963))>,
 <Ballot(Ordet (1955), Winter Light (1963), Resurrection Man (1998), Time Changer (2002))>,
 <Ballot(Winter Light (1963), Ordet (1955), First Reformed (2017), Hunger (2008), Resurrection Man (1998), Time Changer (2002))>,
 <Ballot(Time Changer (2002), First Reformed (2017), Hunger (2008), Winter Light (1963))>,
 <Ballot(Winter Light (1963), Time Changer (2002), First Reformed (2017), Hunger (2008), Ordet (1955), Resurrection Man (1998))>,
 <Ballot(Resurrection Man (1998), Hunger (2008), Winter Light (1963), First Reformed (2017), Ordet (1955), Time Changer (2002))>]

In [22]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [23]:
print(election_result)

ROUND 1
Candidate                  Votes  Status
-----------------------  -------  --------
Winter Light (1963)            2  Hopeful
Hunger (2008)                  1  Hopeful
Time Changer (2002)            1  Hopeful
Ordet (1955)                   1  Hopeful
Resurrection Man (1998)        1  Rejected
First Reformed (2017)          0  Rejected

ROUND 2
Candidate                  Votes  Status
-----------------------  -------  --------
Winter Light (1963)            2  Hopeful
Hunger (2008)                  2  Hopeful
Time Changer (2002)            1  Rejected
Ordet (1955)                   1  Rejected
Resurrection Man (1998)        0  Rejected
First Reformed (2017)          0  Rejected

FINAL RESULT
Candidate                  Votes  Status
-----------------------  -------  --------
Winter Light (1963)            3  Elected
Hunger (2008)                  3  Rejected
Time Changer (2002)            0  Rejected
Ordet (1955)                   0  Rejected
Resurrection Man (1998)        0  Re

In [24]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                       Votes  Status
----------------------------  -------  --------
THE KIDS AREN'T ALRIGHT             2  Hopeful
STRICTLY ROMPS                      1  Hopeful
BREAKING THE IRANIAN EMBARGO        1  Hopeful
CATHOLICS: FRIEND OR FOE?           1  Hopeful
NON-WESTERN SCIFI                   1  Rejected
CLASSIC KUNG FU                     0  Rejected
No Theme/Free-for-all               0  Rejected

ROUND 2
Candidate                       Votes  Status
----------------------------  -------  --------
BREAKING THE IRANIAN EMBARGO        2  Hopeful
THE KIDS AREN'T ALRIGHT             2  Hopeful
STRICTLY ROMPS                      1  Rejected
CATHOLICS: FRIEND OR FOE?           1  Rejected
NON-WESTERN SCIFI                   0  Rejected
CLASSIC KUNG FU                     0  Rejected
No Theme/Free-for-all               0  Rejected

FINAL RESULT
Candidate                       Votes  Status
----------------------------  -------  --------
BREAKING THE IRANIAN 